# Playwright agent

This agent uses Playwright to perform tasks using a web browser. 

In [ ]:
from playwright.sync_api import sync_playwright
from pprint import pprint
import os

# Opper imports
from opperai import fn, start_span
from pydantic import BaseModel, Field

# Environment configuration
os.environ["OPPER_PROJECT"] = "browser-agent"
os.environ["OPPER_DEFAULT_MODEL"] = "openai/gpt4-o"

# Goal definition
goal = "How do I create an API key on the Opper platform? Docs at https://docs.opper.ai"


class Action(BaseModel):
  observation: str
  thought: str
  plan: str
  next_action: str
  action_input: str = Field(
      None,
      description=
      "The appropriate input to the action. For navigation, it should always be the full URL. For return, it should be the answer."
  )


@fn(path="qa/decide_action")
def DetermineAction(goal: str, actions: list, trajectory: list) -> Action:
  """ You are a curious and thoughtful agent in persuit of a goal. Use the actions as you see fit and use the trajectory to see what you have been doing before picking the next action. Try not to repeat steps you have already taken, such as visiting the same pages."""


def print_data(data):
  data = data.dict()
  for key, value in data.items():
    if key in ["thought", "next_action", "action_input"]:
      print(f"{key}: {value}")
  print("-" * 25)


def run_agent():
  actions = ["navigate_and_look", "return"]
  attempts = 15
  trajectory = []

  with sync_playwright() as playwright:
    browser = playwright.chromium.launch(headless=True)
    page = browser.new_page()

    with start_span(name="run"):
      for _ in range(attempts):
        action = DetermineAction(goal=goal,
                                 actions=actions,
                                 trajectory=trajectory)

        print_data(action)

        if action.next_action == "return":
          print("Result:", action.action_input)
          break

        elif action.next_action == "navigate_and_look":
          try:
            page.goto(action.action_input)
            output = {
                "title":
                page.title(),
                "content":
                page.locator('body').inner_text(),
                "url":
                page.url,
                "links": [
                    link.get_attribute("href")
                    for link in page.query_selector_all("a")
                ],
                "forms": [
                    form.get_attribute("action")
                    for form in page.query_selector_all("form")
                ]
            }
          except Exception as e:
            output = {"error": str(e)}

          trajectory.append({
              "action": action.next_action,
              "input": action.action_input,
              "output": output
          })

    browser.close()


if __name__ == "__main__":
  run_agent()
